# ETL — Accidents de la route

Ce notebook montre, pas à pas, comment :
1) se connecter à l’API publique Opendatasoft,
2) récupérer un petit échantillon,
3) paginer pour extraire un volume plus grand,
4) sauvegarder les données brutes en CSV,
5) poser les bases du nettoyage (à faire en équipe).

> **Pourquoi ce format ?**  
> Un notebook est idéal pour apprendre : on alterne **explications** (Markdown) et **code** (Python), et on voit les résultats immédiatement.

In [53]:
# Imports
import pandas as pd
import requests
import time
from pathlib import Path

# Dossiers de sortie
RAW_DIR = Path("../data/raw")
CLEAN_DIR = Path("../data/cleaned")
RAW_DIR.mkdir(parents=True, exist_ok=True)
CLEAN_DIR.mkdir(parents=True, exist_ok=True)


BASE_URL = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records"

SELECT_PARTS = [
    "num_acc",
    "datetime",
    "an",
    "mois",
    "jour",
    "hrmn",
    "lum",
    "agg",
    '"int" as intersection',   # <= réservé → cité + alias
    "atm",
    "col",
    "dep",
    "com",
    "insee",
    "adr",
    "lat",
    '"long" as lon',           # <= réservé → cité + alias
    "surf",
    "circ",
    "nbv",
    "catr",
    "plan",
    "prof",
    "infra",
    "situ",
    "gps",
    "year_georef",
    "dep_name",
    "reg_name",
    "epci_name"
]
SELECT = ", ".join(SELECT_PARTS)


In [54]:
def fetch_page(offset=0, limit=1000):
    params = {
        "select": SELECT,
        "limit": limit,
        "offset": offset,
        "order_by": "datetime"   # champ 'safe' pour trier
    }
    r = requests.get(BASE_URL, params=params, timeout=60)
    r.raise_for_status()
    return r.json().get("results", [])

# Test rapide
sample = fetch_page(0, 10)
df = pd.DataFrame(sample)
df.head(3)

,num_acc,datetime,an,mois,jour,hrmn,lum,agg,intersection,atm,...,catr,plan,prof,infra,situ,gps,year_georef,dep_name,reg_name,epci_name
0,201200037821,2011-12-31T23:15:00+00:00,2012,01,01,00:15,Nuit avec éclairage public allumé,En agglomération,int,Pluie légère,...,Voie Communale,Partie rectiligne,Plat,None,Sur trottoir,Métropole,2015,Seine-et-Marne,Île-de-France,CA Melun Val de Seine
1,201200024058,2011-12-31T23:30:00+00:00,2012,01,01,00:30,Nuit avec éclairage public allumé,En agglomération,int,Pluie légère,...,Voie Communale,Partie rectiligne,Plat,None,Sur chaussée,None,2015,Haute-Garonne,Midi-Pyrénées,Toulouse Métropole
2,201200006244,2011-12-31T23:30:00+00:00,2012,01,01,00:30,Nuit avec éclairage public allumé,En agglomération,int,Pluie légère,...,Route Départementale,Partie rectiligne,Plat,None,Sur accotement,Métropole,2015,Moselle,Lorraine,CC Rives de Moselle


In [55]:
TARGET = 1000
PAGE = 100
DELAY = 0.3

all_rows = []
offset = 0
while offset < TARGET:
    chunk = fetch_page(offset, PAGE)
    if not chunk:
        break
    all_rows.extend(chunk)
    offset += PAGE
    time.sleep(DELAY)

# Sauvegarder un export "brut" pour traçabilité
df_raw = pd.DataFrame(all_rows)
df_raw.to_csv(RAW_DIR / "accidents_sample_raw.csv", index=False)
print(f"{len(df_raw)} lignes enregistrées")


1000 lignes enregistrées


In [56]:
# Règles minimales : garder une clé, dédoublonner
df_clean = df_raw.copy()

# supprimer les lignes sans identifiant
if "numac" in df_clean.columns:
    df_clean = df_clean.dropna(subset=["num_acc"]).drop_duplicates(subset=["num_acc"])



# renommer pour cohérence future
# df_clean = df_clean.rename(columns={
#     "condition_atmospherique": "meteo",
#     "categorie_de_route": "type_route"
# })

len(df_clean), df_clean.head(3)


(1000,
         num_acc                   datetime    an mois jour   hrmn  \
 0  201200037821  2011-12-31T23:15:00+00:00  2012   01   01  00:15   
 1  201200024058  2011-12-31T23:30:00+00:00  2012   01   01  00:30   
 2  201200006244  2011-12-31T23:30:00+00:00  2012   01   01  00:30   
 
                                  lum               agg intersection  \
 0  Nuit avec éclairage public allumé  En agglomération          int   
 1  Nuit avec éclairage public allumé  En agglomération          int   
 2  Nuit avec éclairage public allumé  En agglomération          int   
 
             atm  ...                  catr               plan  prof infra  \
 0  Pluie légère  ...        Voie Communale  Partie rectiligne  Plat  None   
 1  Pluie légère  ...        Voie Communale  Partie rectiligne  Plat  None   
 2  Pluie légère  ...  Route Départementale  Partie rectiligne  Plat  None   
 
              situ        gps year_georef        dep_name       reg_name  \
 0    Sur trottoir  Métropole  

In [58]:
CLEAN_DIR = Path("../data/cleaned")   # bon dossier
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

clean_csv = CLEAN_DIR / "accidents_clean.csv"
df_clean.to_csv(clean_csv, index=False, encoding="utf-8")
clean_csv

WindowsPath('../data/cleaned/accidents_clean.csv')

## Prochaines étapes 

- Générer `dim_time` à partir des dates (ou d’un calendrier)  
- Créer `dim_location` (distinct de `codeinsee`, `departement`, lat/lon si dispo)  
- Créer `dim_conditions` (distinct de `lumiere`, `meteo`, `type_route`, `type_de_collision`)  
- Mapper les IDs (clé naturelle → clé de substitution)  
- Remplir `fact_accident` avec les mesures (`ttue`, `tbg`, `tbl`, `tindm`, `grav`)  
- Écrire `sql/schema.sql` et (option) charger via SQLAlchemy dans SQLite/MySQL/PostgreSQL
